## setup data path final norm

In [1]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

Using TensorFlow backend.


## decide length of the input

In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(pa`th):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)

In [3]:
cal_length(train_path,'ZVa78',train_label_df)

AVG Length:  132.989522417154  file count:  8208  max_len:  5899


## load the data

In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [6]:
data_array_train, data_labels_train = TraverseFiles(train_path,'ZVa78',train_label_df,133)
data_array_test, data_labels_test = TraverseFiles(test_path,'ZVa78',test_label_df,133)

In [7]:
x_train=np.array(data_array_train)
labels_train=np.array(data_labels_train,dtype=int)

In [8]:
x_test=np.array(data_array_test)
labels_test=np.array(data_labels_test,dtype=int)

In [9]:
print(x_train.shape)
print(labels_train.shape)
print(x_test.shape)
print(labels_test.shape)

(8208, 133, 11)
(8208,)
(2052, 133, 11)
(2052,)


## build and train model

In [10]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
lstm.fit(x_train,
    labels_train,
    epochs = 500,
    batch_size=200
)

Epoch 1/500
8208/8208 [==============================] - 5s 648us/step - loss: 0.7127 - accuracy: 0.5166
Epoch 2/500
8208/8208 [==============================] - 5s 647us/step - loss: 0.7129 - accuracy: 0.5050
Epoch 3/500
8208/8208 [==============================] - 5s 655us/step - loss: 0.7037 - accuracy: 0.5192
Epoch 4/500
8208/8208 [==============================] - 5s 662us/step - loss: 0.7025 - accuracy: 0.5130
Epoch 5/500
8208/8208 [==============================] - 5s 663us/step - loss: 0.6992 - accuracy: 0.5290
Epoch 6/500
8208/8208 [==============================] - 5s 665us/step - loss: 0.7020 - accuracy: 0.5291
Epoch 7/500
8208/8208 [==============================] - 5s 666us/step - loss: 0.6972 - accuracy: 0.5251
Epoch 8/500
8208/8208 [==============================] - 6s 676us/step - loss: 0.6941 - accuracy: 0.5253
Epoch 9/500
8208/8208 [==============================] - 6s 685us/step - loss: 0.6941 - accuracy: 0.5302
Epoch 10/500
8208/8208 [==============================]

8208/8208 [==============================] - 5s 654us/step - loss: 0.6765 - accuracy: 0.5835
Epoch 155/500
8208/8208 [==============================] - 5s 657us/step - loss: 0.6774 - accuracy: 0.5786
Epoch 156/500
8208/8208 [==============================] - 5s 655us/step - loss: 0.6767 - accuracy: 0.5808
Epoch 157/500
8208/8208 [==============================] - 5s 655us/step - loss: 0.6749 - accuracy: 0.5810
Epoch 158/500
8208/8208 [==============================] - 5s 654us/step - loss: 0.6761 - accuracy: 0.5799
Epoch 159/500
8208/8208 [==============================] - 5s 656us/step - loss: 0.6764 - accuracy: 0.5843
Epoch 160/500
8208/8208 [==============================] - 5s 654us/step - loss: 0.6763 - accuracy: 0.5833
Epoch 161/500
8208/8208 [==============================] - 5s 645us/step - loss: 0.6754 - accuracy: 0.5826
Epoch 162/500
8208/8208 [==============================] - 5s 640us/step - loss: 0.6769 - accuracy: 0.5797
Epoch 163/500
8208/8208 [==========================

8208/8208 [==============================] - 5s 646us/step - loss: 0.6771 - accuracy: 0.5791
Epoch 307/500
8208/8208 [==============================] - 5s 644us/step - loss: 0.6747 - accuracy: 0.5853
Epoch 308/500
8208/8208 [==============================] - 5s 644us/step - loss: 0.6753 - accuracy: 0.5853
Epoch 309/500
8208/8208 [==============================] - 5s 646us/step - loss: 0.6739 - accuracy: 0.5853
Epoch 310/500
8208/8208 [==============================] - 5s 644us/step - loss: 0.6730 - accuracy: 0.5894
Epoch 311/500
8208/8208 [==============================] - 5s 646us/step - loss: 0.6750 - accuracy: 0.5837
Epoch 312/500
8208/8208 [==============================] - 5s 643us/step - loss: 0.6756 - accuracy: 0.5835
Epoch 313/500
8208/8208 [==============================] - 5s 640us/step - loss: 0.6740 - accuracy: 0.5859
Epoch 314/500
8208/8208 [==============================] - 5s 644us/step - loss: 0.6752 - accuracy: 0.5855
Epoch 315/500
8208/8208 [==========================

8208/8208 [==============================] - 5s 645us/step - loss: 0.6733 - accuracy: 0.5872
Epoch 459/500
8208/8208 [==============================] - 5s 645us/step - loss: 0.6709 - accuracy: 0.5886
Epoch 460/500
8208/8208 [==============================] - 5s 641us/step - loss: 0.6716 - accuracy: 0.5904
Epoch 461/500
8208/8208 [==============================] - 5s 643us/step - loss: 0.6723 - accuracy: 0.5856
Epoch 462/500
8208/8208 [==============================] - 5s 640us/step - loss: 0.6719 - accuracy: 0.5886
Epoch 463/500
8208/8208 [==============================] - 5s 642us/step - loss: 0.6757 - accuracy: 0.5816
Epoch 464/500
8208/8208 [==============================] - 5s 642us/step - loss: 0.6732 - accuracy: 0.5827
Epoch 465/500
8208/8208 [==============================] - 5s 645us/step - loss: 0.6720 - accuracy: 0.5871
Epoch 466/500
8208/8208 [==============================] - 5s 640us/step - loss: 0.6718 - accuracy: 0.5892
Epoch 467/500
8208/8208 [==========================

In [13]:
lstm.save('lstm_model_ZVa78.h5')

## acc on the splitted test set

In [14]:
pred_=lstm.predict_classes(x_test)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(labels_test, pred_,digits=4))
print(accuracy_score(labels_test, pred_))

              precision    recall  f1-score   support

           0     0.6100    0.4936    0.5457      1011
           1     0.5851    0.6936    0.6347      1041

   micro avg     0.5950    0.5950    0.5950      2052
   macro avg     0.5976    0.5936    0.5902      2052
weighted avg     0.5974    0.5950    0.5908      2052

0.5950292397660819


## acc on cleaned test data

In [3]:
train_path = '../../../data/train'
test_path = '../../../data/test'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(cleaned_train_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(cleaned_test_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

In [4]:
cleaned_array_test, cleaned_labels_test = TraverseFiles(test_path,'ZVa78',test_label_df,133)

In [5]:
model = keras.models.load_model('lstm_model_ZVa78.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
cleaned_x=np.array(cleaned_array_test)
cleaned_labels=np.array(cleaned_labels_test,dtype=int)

In [7]:
pred_cleaned=model.predict_classes(cleaned_x)

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(cleaned_labels, pred_cleaned,digits=4))
print(accuracy_score(cleaned_labels, pred_cleaned))

              precision    recall  f1-score   support

           0     0.5428    0.6912    0.6081      3002
           1     0.4895    0.3371    0.3993      2637

   micro avg     0.5256    0.5256    0.5256      5639
   macro avg     0.5162    0.5142    0.5037      5639
weighted avg     0.5179    0.5256    0.5104      5639

0.5256251108352544
